## read result files
These files have been generated with the [reference coreference scorer script](https://github.com/conll/reference-coreference-scorers).

In [12]:
import pandas as pd
import os 
import re
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.1f}'.format

In [13]:
def extract_totals(fname):
    sout = []
    copy = True
    with open(fname) as fin:
        for line in fin.readlines():
            if line.startswith('====== TOTALS ======='):
                copy = True

            if copy:
                sout.append(line)

            if line.startswith('METRIC'):
                copy = False
    return sout

In [14]:
MENTION_RESULTS_REGEX = re.compile(
    r".*Identification of Mentions: Recall: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*Precision: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*F1: ([0-9.]+)%.*", re.DOTALL)

COREF_RESULTS_REGEX = re.compile(
    r".*(?:Coreference|BLANC): Recall: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*Precision: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*F1: ([0-9.]+)%.*", re.DOTALL)

METRIC_REGEX = re.compile(
    r".*METRIC (.*):.*")



def get_results_dict(lines):
    results = {}

    metric = ''
    for s in lines:
        metric_match = re.match(METRIC_REGEX, s)
        if(metric_match):
            metric, = metric_match.groups()
            results[metric] = {}

        mention_match = re.match(MENTION_RESULTS_REGEX, s)
        if(mention_match):
            r, p, f1 = mention_match.groups()
            results[metric]['mention_rec'] = r
            results[metric]['mention_prec'] = p
            results[metric]['mention_f1'] = f1

        coref_match = re.match(COREF_RESULTS_REGEX, s)
        if(coref_match):
            r, p, f1 = coref_match.groups()
            results[metric]['coref_rec'] = r
            results[metric]['coref_prec'] = p
            results[metric]['coref_f1'] = f1
            
    return results

In [15]:
basepath = '/home/dafne/shared/FilterBubble/coref-resolution/pipeline/results'
os.listdir(basepath)

['dev_coref.txt',
 'dev_e2e.txt',
 'dev_entity.txt',
 'news_coref.txt',
 'news_e2e.txt',
 'news_entity.txt',
 'news_goldmentions_coref.txt',
 'news_goldmentions_e2e.txt',
 'news_goldmentions_entity.txt',
 'news_goldparsementions_coref.txt',
 'news_goldparsementions_entity.txt']

In [16]:
results = {}
for fname in os.listdir(basepath):
    expname = fname.split('.')[0]#.split('_')[-1]
    expname = expname.split('_')
    dataset = expname[0]
    tool = expname[-1]
    pipeline = expname[1] if len(expname)>2 else 'endtoend'
    sout = extract_totals(os.path.join(basepath, fname))
    results[(dataset, pipeline, tool)] = get_results_dict(sout)

In [17]:
for exp in results:
    if 'ceafe' in results[exp]:
        results[exp]['conll'] = {}
        for m in ['coref_rec', 'coref_prec','coref_f1']:
            results[exp]['conll'][m] = (float(results[exp]['muc'][m]) + 
                                                 float(results[exp]['bcub'][m]) + 
                                                 float(results[exp]['ceafe'][m]))/3

In [18]:
df_results = pd.DataFrame.from_dict({(*outerKey, innerKey): values for outerKey, innerDict in results.items() 
          for innerKey, values in innerDict.items()}, orient='index')
for c in df_results.columns:
    df_results[c] = df_results[c].astype(float)
    

df_results = df_results.sort_index()
df_results

mention_rec  mention_prec  mention_f1  \
dev  endtoend          coref  bcub          81.8          81.4        81.6   
                              blanc         81.8          81.4        81.6   
                              ceafe         81.8          81.4        81.6   
                              ceafm         81.8          81.4        81.6   
                              conll          nan           nan         nan   
                              muc           81.8          81.4        81.6   
                       e2e    bcub          88.4          88.2        88.3   
                              blanc         88.4          88.2        88.3   
                              ceafe         88.4          88.2        88.3   
                              ceafm         88.4          88.2        88.3   
                              conll          nan           nan         nan   
                              muc           88.4          88.2        88.3   
                       entity bcub          81.8          81.4        81.6   
                              blanc         81.8          81.4        81.6   
                              ceafe         81.8          81.4        81.6   
                              ceafm         81.8          81.4        81.6   
                              conll          nan           nan         nan   
                              muc           81.8          81.4        81.6   
news endtoend          coref  bcub          84.0          86.2        85.0   
                              blanc         84.0          86.2        85.0   
                              ceafe         84.0          86.2        85.0   
                              ceafm         84.0          86.2        85.0   
                              conll          nan           nan         nan   
                              muc           84.0          86.2        85.0   
                       e2e    bcub          91.1          91.6        91.3   
                              blanc         91.1          91.6        91.3   
                              ceafe         91.1          91.6        91.3   
                              ceafm         91.1          91.6        91.3   
                              conll          nan           nan         nan   
                              muc           91.1          91.6        91.3   
                       entity bcub          84.0          86.2        85.0   
                              blanc         84.0          86.2        85.0   
                              ceafe         84.0          86.2        85.0   
                              ceafm         84.0          86.2        85.0   
                              conll          nan           nan         nan   
                              muc           84.0          86.2        85.0   
     goldmentions      coref  bcub          86.6          89.3        88.0   
                              blanc         86.6          89.3        88.0   
                              ceafe         86.6          89.3        88.0   
                              ceafm         86.6          89.3        88.0   
                              conll          nan           nan         nan   
                              muc           86.6          89.3        88.0   
                       e2e    bcub          91.2         100.0        95.4   
                              blanc         91.2         100.0        95.4   
                              ceafe         91.2         100.0        95.4   
                              ceafm         91.2         100.0        95.4   
                              conll          nan           nan         nan   
                              muc           91.2         100.0        95.4   
                       entity bcub          86.6          89.3        88.0   
                              blanc         86.6          89.3        88.0   
                              ceafe         86.6          8

In [19]:
df_results_swapped = df_results.swaplevel().sort_index()

In [20]:
# Show mention identication scores, sorted on f1
mention_scores = df_results_swapped.loc[(slice(None), slice(None),'muc',slice(None)), ['mention_rec', 'mention_prec', 'mention_f1']]
mention_scores = mention_scores.sort_values('mention_f1', ascending=False)
mention_scores = mention_scores.sort_index(level=[0,1], sort_remaining=False)
mention_scores

mention_rec  mention_prec  mention_f1
dev  endtoend          muc e2e            88.4          88.2        88.3
                           coref          81.8          81.4        81.6
                           entity         81.8          81.4        81.6
news endtoend          muc e2e            91.1          91.6        91.3
                           coref          84.0          86.2        85.0
                           entity         84.0          86.2        85.0
     goldmentions      muc e2e            91.2         100.0        95.4
                           coref          86.6          89.3        88.0
                           entity         86.6          89.3        88.0
     goldparsementions muc coref          94.5          95.8        95.2
                           entity         94.5          95.8        95.2

In [22]:
sorted_scores = df_results_swapped.sort_values('coref_f1',ascending=False)
sorted_scores = sorted_scores.sort_index(level=[0,1,2], sort_remaining=False)[['coref_rec', 'coref_prec', 'coref_f1']]
sorted_scores

coref_rec  coref_prec  coref_f1
dev  endtoend          bcub  e2e          71.9        81.8      76.5
                             entity       65.7        68.5      67.0
                             coref        64.9        67.6      66.2
                       blanc e2e          61.9        72.5      66.2
                             coref        51.8        51.4      51.6
                             entity       48.1        41.5      43.9
                       ceafe e2e          80.3        69.9      74.7
                             entity       66.5        63.4      64.9
                             coref        65.8        63.3      64.5
                       ceafm e2e          72.6        72.5      72.6
                             entity       62.4        62.1      62.2
                             coref        61.5        61.1      61.3
                       conll e2e          67.2        75.6      70.3
                             entity       59.5        61.3      60.3
                             coref        58.1        59.4      58.7
                       muc   e2e          49.4        75.1      59.6
                             entity       46.2        51.9      48.9
                             coref        43.5        47.5      45.4
news endtoend          bcub  e2e          78.3        87.4      82.6
                             entity       68.5        78.7      73.3
                             coref        69.7        74.4      71.9
                       blanc e2e          61.9        78.7      68.0
                             coref        54.1        59.3      56.5
                             entity       50.8        61.2      55.0
                       ceafe e2e          86.6        77.2      81.6
                             entity       75.4        68.6      71.8
                             coref        71.8        69.5      70.6
                       ceafm e2e          79.3        79.7      79.5
                             entity       69.2        70.9      70.0
                             coref        68.0        69.6      68.8
                       conll e2e          72.2        81.9      75.8
                             entity       60.7        69.3      64.0
                             coref        61.2        65.2      63.0
                       muc   e2e          51.7        81.0      63.1
                             entity       38.2        60.6      46.9
                             coref        42.3        51.9      46.6
     goldmentions      bcub  e2e          78.4        94.8      85.8
                             entity       71.0        82.3      76.2
                             coref        71.9        78.2      74.9
                       blanc e2e          61.4        84.7      70.6
                             coref        55.8        63.8      59.4
                             entity       53.6        64.9      58.2
                       ceafe e2e          86.2        85.5      85.8
                             entity       78.3        71.5      74.8
                             coref        75.1        73.4      74.2
                       ceafm e2e          79.0        86.7      82.7
                             entity       71.4        73.7      72.5
                             coref        70.6        72.9      71.7
                       conll e2e          72.1        86.8      78.2
                             entity       62.8        72.5      66.5
                             coref        63.3        68.1      65.5
                       muc   e2e          51.7        80.2      62.9
                             entity       39.3        63.6      48.5
                             coref        43.0        52.7      47.3
     goldparsementions bcub  entity       79.0        88.0      83.2
                             coref        79.9        84.3      82.0
                       blanc coref        64.9        70.4      67.5
                             entity